In [1]:
!pip install yfinance


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pd.read_csv('etf_funds/ETFs.csv')

In [3]:
fund_names = df["fund_symbol"].values

In [30]:
# funds_historical_data = yf.download(list(fund_names), period='1y')
# funds_historical_data["Adj Close"].to_csv("etf_funds/historical_data.csv")

[*********************100%***********************]  2310 of 2310 completed

10 Failed downloads:
- GCOW: No data found for this date range, symbol may be delisted
- SHLD: No data found for this date range, symbol may be delisted
- VTRN: No data found for this date range, symbol may be delisted
- CHIL: No data found for this date range, symbol may be delisted
- AGGP: No data found for this date range, symbol may be delisted
- TFIV: No data found for this date range, symbol may be delisted
- TFLT: No data found for this date range, symbol may be delisted
- FRAK: No data found for this date range, symbol may be delisted
- RFUN: No data found for this date range, symbol may be delisted
- OLD: No data found for this date range, symbol may be delisted


In [4]:
file_df = pd.read_csv("etf_funds/historical_data.csv")

In [5]:
fund_names = list(file_df.columns[1:].values)
fund_returns = []
fund_vol  = []
fund_sharpe = []


for fund in fund_names:
    fund_list = file_df[fund].values
    fund_return = (fund_list[-1] - fund_list[0]) / fund_list[0]
    fund_volatility = file_df[fund].pct_change().std() * np.sqrt(252)
    
    fund_returns.append(fund_return)
    fund_vol.append(fund_volatility)
    fund_sharpe.append(fund_return / fund_volatility)
 

In [6]:
df_final = pd.DataFrame(data={ 
    "Symbol": fund_names, 
    "1Y Return": fund_returns, 
    "1Y Volatility": fund_vol, 
    "1Y Sharpe": fund_sharpe 
})

In [7]:
# Removendo fundos que já encerraram as operações
df_final = df_final.dropna()

In [8]:
df_final

,Symbol,1Y Return,1Y Volatility,1Y Sharpe
0,AAA,-0.015278,0.022634,-0.675021
1,AAAU,-0.078962,0.141250,-0.559023
2,AADR,-0.251947,0.250346,-1.006395
3,AAXJ,-0.337440,0.232847,-1.449193
4,ABEQ,-0.009397,0.136445,-0.068873
...,...,...,...,...
2305,ZHDG,-0.162616,0.136425,-1.191983
2306,ZIG,-0.139991,0.225021,-0.622125
2307,ZIVZF,-0.234234,0.554628,-0.422327
2308,ZROZ,-0.433651,0.276078,-1.570754


In [9]:
# fund_holdings = {}
# unique_assets = []
# error_list = []

# for index, fund in enumerate(list(df_final["Symbol"].values)):
#     try:
#         asset = yf.Ticker(fund)
#         holdings_info = asset.get_info()['holdings']
#         holdings = {}
        
#         for holding in holdings_info:
#             holdings[holding["holdingName"]] = holding["holdingPercent"]
#             unique_assets.append(holding["holdingName"])
        
#         fund_holdings[fund] = holdings
#         if (index % 100 == 0):
#             print(index)
#     except:
#         error_list.append(fund)

In [10]:
# f = open('fund_holdings.pckl', 'wb')
# pickle.dump(fund_holdings, f)
# f.close()

# f = open('unique_assets.pckl', 'wb')
# pickle.dump(list(set(unique_assets)), f)
# f.close()

# f = open('error_list.pckl', 'wb')
# pickle.dump(error_list, f)
# f.close()

In [15]:
f = open('error_list.pckl', 'rb')
error_list = pickle.load(f)
f.close()

f = open('unique_assets.pckl', 'rb')
unique_assets = pickle.load(f)
f.close()

f = open('fund_holdings.pckl', 'rb')
fund_holdings = pickle.load(f)
f.close()

In [16]:
matrix = pd.DataFrame(columns=list(set(unique_assets)), index=fund_holdings.keys())
matrix.fillna(0, inplace=True)

for fund in fund_holdings:
    for asset in fund_holdings[fund]:
        matrix.loc[fund, asset] = fund_holdings[fund][asset]

In [33]:
# Euclidean distance between two vectors for all funds
from scipy.spatial.distance import pdist, squareform
distances = pd.DataFrame(squareform(pdist(matrix, 'euclidean')), columns=matrix.index, index=matrix.index)

distances

,AAA,AAAU,AADR,AAXJ,ABEQ,ACES,ACIO,ACSI,ACTV,ACVF,...,YLDE,YMAR,YOLO,YPS,YXI,YYY,ZHDG,ZIG,ZROZ,ZSL
AAA,0.000000,1.012001,0.205556,0.195133,0.230977,0.223306,0.203392,0.229159,0.181673,0.177798,...,0.189402,0.971054,0.390675,0.157000,0.479807,0.208836,0.155390,0.219455,0.163359,0.972715
AAAU,1.012001,0.000000,1.009013,1.006941,1.014497,1.012778,1.008574,1.014085,1.004420,1.003726,...,1.005846,1.385208,1.062300,1.000251,1.098211,1.009686,1.000000,1.011936,1.001269,1.386372
AADR,0.205556,1.009013,0.000000,0.178992,0.217512,0.209348,0.187961,0.215580,0.164214,0.159917,...,0.172726,0.967940,0.382868,0.136419,0.473472,0.193839,0.134563,0.205236,0.143692,0.969606
AAXJ,0.195133,1.006941,0.178992,0.000000,0.207690,0.199124,0.176502,0.205666,0.150963,0.146277,...,0.160180,0.965780,0.377375,0.120141,0.469041,0.182749,0.118029,0.194796,0.128340,0.967450
ABEQ,0.230977,1.014497,0.217512,0.207690,0.000000,0.234358,0.215468,0.239942,0.195098,0.181068,...,0.202315,0.973655,0.397096,0.172358,0.485049,0.220615,0.170893,0.230692,0.178170,0.975311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YYY,0.208836,1.009686,0.193839,0.182749,0.220615,0.212569,0.191543,0.218710,0.168301,0.164111,...,0.176616,0.968642,0.384639,0.141312,0.474905,0.000000,0.139522,0.208521,0.148346,0.970306
ZHDG,0.155390,1.000000,0.134563,0.118029,0.170893,0.160372,0.131233,0.168427,0.094122,0.086406,...,0.108290,0.958541,0.358442,0.022424,0.453948,0.139522,0.000000,0.154966,0.050400,0.960223
ZIG,0.219455,1.011936,0.205236,0.194796,0.230692,0.223011,0.203068,0.228871,0.181310,0.177427,...,0.189054,0.970987,0.390507,0.156580,0.479670,0.208521,0.154966,0.000000,0.162956,0.972647
ZROZ,0.163359,1.001269,0.143692,0.128340,0.178170,0.168105,0.140579,0.175806,0.106767,0.100031,...,0.119444,0.959865,0.361968,0.055163,0.456737,0.148346,0.050400,0.162956,0.000000,0.961545


In [35]:
distances.std().mean()

69.09179410463096

In [32]:
media_jrg = distances.mean().mean() - 1.5 * (distances.std().mean())
media_jrg

-98.4156211026171